# regresión lineal regular 

Datos sociodemográficos y de recursos de salud a nivel de condado de EE. UU. (2018-2019)
Se han recopilado datos sociodemográficos y de recursos de salud por condado en los Estados Unidos y queremos descubrir si existe alguna relación entre los recursos sanitarios y los datos sociodemográficos.

Para ello, es necesario que establezcas una variable objetivo (relacionada con la salud) para llevar a cabo el análisis.



1. Cargo el dataset COMPLETO: con todas sus filas y columnas.
2. Divido el dataset en TRAIN y TEST. Me guardo los IDs de TRAIN y los IDs de TEST.
3. Hago un feature selection (SelectKBest) y me quedo con el 30% de las predictoras usando el dataset de TRAIN. Si tenía 108, me quedo con 36
4. Una vez el modelo de selección me ha dado las 36 mejores, filtro TRAIN y TEST y me quedo con esas variables (incluyendo la target)
5. Junto TRAIN y TEST para hacer un EDA completo
6. En el EDA, en el paso del análisis multivariante, analizo si todas las variables tienen correlación, o si hay algunas que podría descartar. En el caso de que
pudiera descartar, aplicaría de nuevo un feature selection (SelectKBest) para eliminarlas.
7. Listo para entrenar mi regresión lineal
8. Listo para entrenar mi L1 (regresión lineal regularizada de Lasso)

paso 1: cargar el dataset completo 

In [17]:
import seaborn as sns

import pandas as pd

total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")



print (total_data.head())

total_data.info()


   fips  TOT_POP    0-9  0-9 y/o % of total pop  19-Oct  \
0  1001    55601   6787               12.206615    7637   
1  1003   218022  24757               11.355276   26913   
2  1005    24881   2732               10.980266    2960   
3  1007    22400   2456               10.964286    2596   
4  1009    57840   7095               12.266598    7570   

   10-19 y/o % of total pop  20-29  20-29 y/o % of total pop  30-39  \
0                 13.735364   6878                 12.370281   7089   
1                 12.344167  23579                 10.814964  25213   
2                 11.896628   3268                 13.134520   3201   
3                 11.589286   3029                 13.522321   3113   
4                 13.087828   6742                 11.656293   6884   

   30-39 y/o % of total pop  ...  COPD_number  diabetes_prevalence  \
0                 12.749771  ...         3644                 12.9   
1                 11.564429  ...        14692                 12.0   
2       

5 filas, 108 columnas...

paso 2: divido el dataset en train y en test 
me guardo los ids de train y test 

In [19]:
total_data.columns



Index(['fips', 'TOT_POP', '0-9', '0-9 y/o % of total pop', '19-Oct',
       '10-19 y/o % of total pop', '20-29', '20-29 y/o % of total pop',
       '30-39', '30-39 y/o % of total pop',
       ...
       'COPD_number', 'diabetes_prevalence', 'diabetes_Lower 95% CI',
       'diabetes_Upper 95% CI', 'diabetes_number', 'CKD_prevalence',
       'CKD_Lower 95% CI', 'CKD_Upper 95% CI', 'CKD_number',
       'Urban_rural_code'],
      dtype='object', length=108)

In [20]:
total_data[["Total Population", "TOT_POP"]]

,Total Population,TOT_POP
0,55036,55601
1,203360,218022
2,26201,24881
3,22580,22400
4,57667,57840
...,...,...
3135,44527,43051
3136,22923,23081
3137,20758,20299
3138,8253,7885


Entre Total Population y TOT_POP escojo Total Pòpulation como la target 

HAY columnas de object, las transformo en integer 

In [21]:
import pandas as pd



# Seleccionar las columnas de tipo 'object'
object_columns = total_data.select_dtypes(include=['object'])

# Convertir las columnas seleccionadas a tipo 'integer', reemplazando valores no numéricos con NaN
for column in object_columns:
    total_data[column] = pd.to_numeric(total_data[column], errors='coerce')


In [23]:
total_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(63), int64(45)
memory usage: 2.6 MB


Reemplazo NaN

In [26]:
total_data.isna().sum()

fips                      0
TOT_POP                   0
0-9                       0
0-9 y/o % of total pop    0
19-Oct                    0
                         ..
CKD_prevalence            0
CKD_Lower 95% CI          0
CKD_Upper 95% CI          0
CKD_number                0
Urban_rural_code          0
Length: 108, dtype: int64

In [27]:
import pandas as pd



# Obtener una lista de columnas que tienen al menos un NaN
columns_with_nan = total_data.columns[total_data.isna().any()].tolist()

print("Columnas con valores NaN:")
print(columns_with_nan)


Columnas con valores NaN:
['COUNTY_NAME', 'STATE_NAME']


factorizo estas columnas, necesario para dividir en TRain y Test y elimino las no factorizadas 

In [28]:
print(total_data["COUNTY_NAME"])



0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3135   NaN
3136   NaN
3137   NaN
3138   NaN
3139   NaN
Name: COUNTY_NAME, Length: 3140, dtype: float64


In [29]:
import pandas as pd

total_data["COUNTY_NAME_N"] = pd.factorize( total_data["COUNTY_NAME"])[0]
total_data["STATE_NAME_N"] = pd.factorize( total_data["STATE_NAME"])[0]

columns_to_drop = ['COUNTY_NAME', 'STATE_NAME']
total_data = total_data.drop(columns=columns_to_drop)

# Obtener una lista de columnas que tienen al menos un NaN
columns_with_nan = total_data.columns[total_data.isna().any()].tolist()

print("Columnas con valores NaN:")
print(columns_with_nan)




Columnas con valores NaN:
[]


ahora divido en train y test 

In [30]:

import pandas as pd
from sklearn.model_selection import train_test_split



# Dividir total_data en X (características) e y (etiquetas)
X = total_data.drop(columns=['Total Population'])  
y = total_data['Total Population']

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train.head()


,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code,COUNTY_NAME_N,STATE_NAME_N
1292,26127,26625,3221,12.097653,3463,13.006573,2922,10.974648,2829,10.625352,...,12.6,14.9,2823,3.8,3.5,4.1,771,6,-1,-1
2302,42121,51266,5272,10.283619,5751,11.217961,5137,10.020286,5341,10.418211,...,11.9,14.2,5416,3.5,3.2,3.8,1454,5,-1,-1
761,18133,37779,3915,10.362900,5118,13.547209,6202,16.416528,4363,11.548744,...,11.2,13.1,3698,2.9,2.7,3.1,871,2,-1,-1
2194,40131,91984,11163,12.135806,12646,13.748043,11595,12.605453,11357,12.346712,...,10.4,12.0,7913,3.0,2.8,3.2,2118,3,-1,-1
1241,26025,134487,16698,12.416070,17666,13.135842,17281,12.849569,15993,11.891856,...,11.7,13.4,12987,3.4,3.2,3.6,3490,4,-1,-1


nos guardamos el id de train y test 

In [31]:
train_indices = X_train.index
test_indices = X_test.index


Hago un feature selection (SelectKBest) y me quedo con el 30% de las predictoras usando el dataset de TRAIN. Si tenía 108, me quedo con 36

filtro para incluir estas 36 variables y el target 

In [34]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression

# Supongamos que seleccionamos las mejores 36 características
k_best_features = 36
selection_model = SelectKBest(score_func=f_regression, k=k_best_features)
X_train_selected = selection_model.fit_transform(X_train, y_train)

# Obtener la máscara de características seleccionadas (índices True/False)
selected_mask = selection_model.get_support()

# Obtener los nombres de las características seleccionadas
selected_feature_names = X_train.columns[selected_mask]

# Filtrar las características seleccionadas en X_test
X_test_selected = X_test[selected_feature_names]

# Convertir 'X_train_selected' a un DataFrame de Pandas con los nombres de las características seleccionadas
X_train_selected_total = pd.DataFrame(X_train_selected, columns=selected_feature_names)

# Crear una copia del DataFrame 'X_train_selected_total' para agregar 'y_train' como columna
X_train_selected_with_target = X_train_selected_total.copy()

# Ahora 'X_train_selected_with_target' contiene las características seleccionadas de entrenamiento

# Concatenar 'X_train_selected_with_target' y 'X_test_selected' en un solo DataFrame llamado 'total_data'
total_data = pd.concat([X_train_selected_with_target, X_test_selected], axis=0)

# Agregar la variable objetivo 'POP_TOT' solo para las etiquetas de prueba en 'total_data'
total_data['Total Population'] = pd.concat([y_train, y_test], axis=0)

# Verificar el tamaño del DataFrame resultante
print("Tamaño de total_data:", total_data.shape)

# Obtener una lista de columnas que tienen al menos un NaN
columns_with_nan = total_data.columns[total_data.isna().any()].tolist()

print("Columnas con valores NaN:")
print(columns_with_nan)

total_data.columns


Tamaño de total_data: (3140, 37)
Columnas con valores NaN:
[]


Index(['TOT_POP', '0-9', '19-Oct', '20-29', '30-39', '40-49', '50-59', '60-69',
       '70-79', '80+', 'White-alone pop', 'Two or more races pop',
       'POP_ESTIMATE_2018', 'Less than a high school diploma 2014-18',
       'High school diploma only 2014-18',
       'Some college or associate's degree 2014-18',
       'Bachelor's degree or higher 2014-18', 'POVALL_2018',
       'Civilian_labor_force_2018', 'Employed_2018', 'Unemployed_2018',
       'Total nurse practitioners (2019)', 'Total physician assistants (2019)',
       'Total Hospitals (2019)', 'Internal Medicine Primary Care (2019)',
       'Family Medicine/General Practice Primary Care (2019)',
       'Total Specialist Physicians (2019)', 'ICU Beds_x',
       'Population Aged 60+', 'county_pop2018_18 and older',
       'anycondition_number', 'Obesity_number', 'Heart disease_number',
       'COPD_number', 'diabetes_number', 'CKD_number', 'Total Population'],
      dtype='object')

salvo el nuevo dataset en data/processed

In [35]:
total_data.info


<bound method DataFrame.info of        TOT_POP      0-9   19-Oct    20-29    30-39    40-49    50-59    60-69  \
0      26625.0   3221.0   3463.0   2922.0   2829.0   2831.0   3831.0   3860.0   
1      51266.0   5272.0   5751.0   5137.0   5341.0   5880.0   7885.0   8271.0   
2      37779.0   3915.0   5118.0   6202.0   4363.0   4451.0   5107.0   4349.0   
3      91984.0  11163.0  12646.0  11595.0  11357.0  11444.0  12774.0  10791.0   
4     134487.0  16698.0  17666.0  17281.0  15993.0  15845.0  17982.0  16909.0   
...        ...      ...      ...      ...      ...      ...      ...      ...   
1351   28494.0   3569.0   3820.0   3026.0   3491.0   3504.0   4147.0   3590.0   
670     5740.0    592.0    669.0    593.0    631.0    616.0    877.0    892.0   
2283   40968.0   4167.0   5072.0   4980.0   4758.0   5055.0   5804.0   5646.0   
2482   97605.0  12083.0  13272.0  13493.0  11316.0  11297.0  13078.0  12061.0   
2650   45641.0   5152.0   6468.0   4549.0   5125.0   6056.0   6405.0   5949.0

In [36]:
# Obtener una lista de columnas que tienen al menos un NaN
columns_with_nan = total_data.columns[total_data.isna().any()].tolist()

print("Columnas con valores NaN:")
print(columns_with_nan)

Columnas con valores NaN:
[]


In [56]:
total_data.to_csv("/workspaces/joseVent-linearregular/data/processed/total_data.csv", index=False)